In [1]:
import sys
sys.path.append("../src/")

In [4]:
import pandas as pd
df = pd.read_csv("../data/raw/Enriched_Set_60percent_similarity.csv")
df.head()

,SMILES,Target,isActive,SELFIES,Seq
0,Cn1c(=O)c2[nH]cnc2n(C)c1=O,aldh1a1,1,[C][N][C][=Branch1][C][=O][C][NH1][C][=N][C][=...,MSSSGTPDLPVLLTDLKIQYTKIFINNEWHDSVSGKKFPVFNPATE...
1,Oc1cc(Sc2nc[nH]n2)c(O)n1-c1ccccc1,aldh1a1,1,[O][C][=C][C][Branch1][=Branch2][S][C][N][=C][...,MSSSGTPDLPVLLTDLKIQYTKIFINNEWHDSVSGKKFPVFNPATE...
2,O=C(c1ccccc1)c1c(-c2ccccc2)oc(O)c1O,aldh1a1,1,[O][=C][Branch1][=Branch2][C][=C][C][=C][C][=C...,MSSSGTPDLPVLLTDLKIQYTKIFINNEWHDSVSGKKFPVFNPATE...
3,COc1ccc(-n2c(O)cc(Sc3n[nH]c(-c4ccc(C)cc4)n3)c2...,aldh1a1,1,[C][O][C][=C][C][=C][Branch2][Ring2][Branch1][...,MSSSGTPDLPVLLTDLKIQYTKIFINNEWHDSVSGKKFPVFNPATE...
4,CCc1ccc(C2NC(=S)N=C3CCCC(=O)C32)cc1,aldh1a1,1,[C][C][C][=C][C][=C][Branch2][Ring1][Branch1][...,MSSSGTPDLPVLLTDLKIQYTKIFINNEWHDSVSGKKFPVFNPATE...


In [5]:
import selfies as sf

# Decode SELFIES to get the actual structure and calculate length
df['decoded_SELFIES'] = df['SELFIES'].apply(lambda x: sf.decoder(x))
df['SELFIES_length'] = df['decoded_SELFIES'].apply(len)

# Calculate length of protein sequences directly
df['Seq_length'] = df['Seq'].apply(len)

# Compute statistics
selfies_stats = {
    'max_length': df['SELFIES_length'].max(),
    'min_length': df['SELFIES_length'].min(),
    'avg_length': df['SELFIES_length'].mean()
}

seq_stats = {
    'max_length': df['Seq_length'].max(),
    'min_length': df['Seq_length'].min(),
    'avg_length': df['Seq_length'].mean()
}

print("SELFIES Stats:", selfies_stats)
print("Protein Sequence Stats:", seq_stats)

SELFIES Stats: {'max_length': 347, 'min_length': 2, 'avg_length': 55.93296800667567}
Protein Sequence Stats: {'max_length': 776, 'min_length': 209, 'avg_length': 456.56274356305323}


In [6]:
# Extract unique symbols from SELFIES
def extract_unique_selfies_symbols(selfies_string):
    return sf.split_selfies(selfies_string)

# Apply the function to each SELFIES string and accumulate the unique symbols in a set
unique_symbols = set()
for _, row in df.iterrows():
    symbols = extract_unique_selfies_symbols(row['SELFIES'])
    unique_symbols.update(symbols)

# Count the unique symbols. Add 3 for <SOS>, <EOS>, and <PAD> special tokens.
vocab_size = len(unique_symbols) + 3

print(f"SELFIES Vocabulary Size: {vocab_size}")

SELFIES Vocabulary Size: 109


In [ ]:
#So our SELFIES final Vocab size is 112 with PAD, EOS and SOS.
# Protein final vocab = 20 AA's plus PAD, EOS, SOS = 23.